In [45]:
import sys
import os
from shapely import speedups
speedups.disable()
from shapely.geometry import shape
import fiona
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from datetime import datetime
from mpl_toolkits.axes_grid1 import make_axes_locatable
from zipfile import ZipFile
from folium.plugins import MarkerCluster
from folium import plugins
from shapely.ops import nearest_points
from folium.plugins import FloatImage
import networkx as nx
from shapely import wkt
import branca
import json
from geopandas import GeoDataFrame
from shapely.geometry import Point,LineString
import math
import warnings
warnings.filterwarnings("ignore")
pd.set_option('max_colwidth', 800)
import requests
from datetime import datetime, timedelta
#!pip install frvcpy
from frvcpy.translator import translate
from frvcpy.solver import Solver
from frvcpy import solver
import time
from numpy import inf

# Preperation

In [46]:
key = #OS API_KEY
weather_api_key = #Weather_API_KEY
layer = 'Light_3857'
zxy_path = 'https://api.os.uk/maps/raster/v1/zxy/{}/{{z}}/{{x}}/{{y}}.png?key={}'.format(layer, key)

In [47]:
def is_stations_close_poly(dataframe_name, df, id, max_distance):
    scotland_stations_geo_last[dataframe_name] = ""
    scotland_stations_geo_last[dataframe_name] = scotland_stations_geo_last[dataframe_name].astype(object)
    temper = []
    for index, v in scotland_stations_geo_last.iterrows():
        tempor_df=df[(scotland_stations_geo_last.latitude[index]+0.1 > df.latitude) &(scotland_stations_geo_last.latitude[index]-0.1 < df.latitude)
          &(scotland_stations_geo_last.longitude[index]+0.1 > df.longitude)& (scotland_stations_geo_last.longitude[index]-0.1 < df.longitude)].reset_index(drop=True)
        point= v.geometry
        for j , k in tempor_df.iterrows(): 
            near_p = nearest_points(k.geometry,point)
            if Haversine(near_p[0].coords[0], point.coords[0]).km <= max_distance:
                temper.append(tempor_df.loc[j,id])
        if len(temper) != 0: 
            scotland_stations_geo_last[dataframe_name][index] = temper
        temper = []
        print(index)
    return temper
#is_stations_close_poly('closest_road',scotland_road_filtered,'id', 1)
#scotland_stations_geo_last.to_csv('C:\\Users\\yusuf\\Downloads\\Dissertation Code\\Scotland_Stations_Closest_Road_Last.csv', index=None, header=True)

In [48]:
#Change for optimization 
def check_postcode_validation(postcode):
    req=requests.get(f"https://api.postcodes.io/postcodes/{postcode}/validate").json()
    return req.get('result', [])
def call_Weather_Data_WRT_Day(day, post_or_loc, weather_api_key,hour):
    req =requests.get(f"http://api.weatherapi.com/v1/forecast.json?key={weather_api_key}&q={post_or_loc}&days={day}&aqi=no&alerts=no&hour={hour}").json()
    weather_dict = {}
    weather_dict["name"] = req['location']["name"]
    weather_dict["latitude"] =req['location']["lat"]
    weather_dict["longitude"] = req['location']["lon"]
    weather_dict['current_temperature'] = req['current']['temp_c']
    weather_dict['current_temperature_f'] = req['current']['temp_f']
#     weather_dict['date_forecast'] = req['forecast']['forecastday'][day]['date']
#     weather_dict['forecasted_temp'] = req['forecast']['forecastday'][day]['hour'][0]['temp_c']
#     forecasted_temp_f = req['forecast']['forecastday'][day]['hour'][0]['temp_c']
#     weather_dict['chance_of_rain']= req['forecast']['forecastday'][day]['hour'][0]['chance_of_rain']
#     weather_dict['chance_of_snow'] = req['forecast']['forecastday'][day]['hour'][0]['chance_of_snow']
    return weather_dict

In [49]:
def call_OSMR_route(start_point,end_point):
    r = requests.get(f"http://router.project-osrm.org/route/v1/car/{start_point.x},{start_point.y};{end_point.x},{end_point.y}?overview=full&geometries=geojson").json()
    #import polyline
    # polyline.decode(route_1['geometry'],geojson=True) for geometries object on API link change to geometries=polyline
    return r  
def call_OSMR_route_add_stop(start_point,end_point, **stops):
    base = f"http://router.project-osrm.org/route/v1/car/{start_point.x},{start_point.y};"
    options = f"{end_point.x},{end_point.y}?overview=full&geometries=geojson"
    nodes = "" 
    for key, val in stops.items():
        for k , value in  val.items():
            nodes = nodes + f"{value.x},{value.y};"
        full_http = base+nodes+options
        print(full_http)
    rq = requests.get(full_http).json()
    return rq
def call_OSMR_optimal_route(elements):
    base = "http://router.project-osrm.org/route/v1/car/"
    chck = len(elements)
    a = 0 
    for i in selected_elements:
        base = base + str(i[0]) + "," +str(i[1]) 
        if a < chck-1:
            base = base +";"
        a += 1
    complete_html = base + "?overview=full&geometries=geojson"
    r = requests.get(complete_html)
    #print(r.json())
    route = json.loads(r.content)["routes"][0]
    return route["distance"],route["geometry"]['coordinates']
def call_OSMR_route_optimise(start_point,end_point):
    r = requests.get(f"http://router.project-osrm.org/route/v1/car/{start_point.x},{start_point.y};{end_point.x},{end_point.y}?overview=full&geometries=geojson")
    #print(r.json())
    route = json.loads(r.content)["routes"][0]
    return (route["distance"], route["duration"],route["geometry"] )
def Elevation(points):
    url_head = "https://api.open-elevation.com/api/v1/lookup?locations="
    #url_opentodata = "https://api.opentopodata.org/v1/test-dataset?locations="
    url_comp=url_head + points
    elevations = requests.get(url_comp).json()
    return [x['elevation'] for x in elevations['results']]
def call_distance_matrix_openroute(node_list):
    body = {"locations":node_list,"metrics":["distance","duration"]}
    headers = {
        'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
        'Authorization': #openroutesource API_KEY,
        'Content-Type': 'application/json; charset=utf-8'
    }
    call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)
    print(call.status_code, call.reason)
    ret = call.json()
    #print(call.text)
    DistMatrix_pd_duration = pd.DataFrame(ret['durations']).reset_index(drop =False)
    DistMatrix_pd_distance = pd.DataFrame(ret['distances']).reset_index(drop =False)


    df_melt_distance=pd.melt(DistMatrix_pd_distance, id_vars=['index'], value_vars=DistMatrix_pd_distance.columns.values.tolist()[1:],
            var_name='Dest', value_name='Distance')
    df_melt_duration=pd.melt(DistMatrix_pd_duration, id_vars=['index'], value_vars=DistMatrix_pd_duration.columns.values.tolist()[1:],
            var_name='Dest', value_name='Duration')

    df_melt_distance = df_melt_distance.rename(columns={'index': 'Origin'})
    df_melt_duration = df_melt_duration.rename(columns={'index': 'Origin'})
    df_melt_duration["Distance"] = df_melt_distance["Distance"]
    
    return df_melt_duration

In [50]:
class Haversine:
    '''
    use the haversine class to calculate the distance between
    two lon/lat coordnate pairs.
    output distance available in kilometers, meters, miles, and feet.
    example usage: Haversine([lon1,lat1],[lon2,lat2]).feet
    
    '''
    def __init__(self,coord1,coord2):
        lon1,lat1=coord1
        lon2,lat2=coord2
        
        R=6371000                               # radius of Earth in meters
        phi_1=math.radians(lat1)
        phi_2=math.radians(lat2)

        delta_phi=math.radians(lat2-lat1)
        delta_lambda=math.radians(lon2-lon1)

        a=math.sin(delta_phi/2.0)**2+\
           math.cos(phi_1)*math.cos(phi_2)*\
           math.sin(delta_lambda/2.0)**2
        c=2*math.atan2(math.sqrt(a),math.sqrt(1-a))
        
        self.meters=R*c                         # output distance in meters
        self.km=self.meters/1000.0              # output distance in kilometers
        self.miles=self.meters*0.000621371      # output distance in miles
        self.feet=self.miles*5280               # output distance in feet

if __name__ == "__Haversine__":
    main()

In [51]:
def fancy_html_close(row, df):
    i = row
    
    Number_of_Vehicles = df['Number_of_Connector'].iloc[i]
    Connector_Type = df['Connector_Type'].iloc[i]   
    Connector_KW = df['Connector_KW'].iloc[i]
    Post_code = df['post_code'].iloc[i]
    if len(df["Worship_osm_poly_elem"].iloc[i])==0:
        Worship_OSM = ""
    else:
        Worship_OSM = Worship_osm_poly_elem[Worship_osm_poly_elem['osm_id'].isin(df["Worship_osm_poly_elem"].iloc[i])][["fclass", "name"]][:5].values.tolist()
    if len(df["landuse_osm_poly"].iloc[i])==0:
        Landuse_OSM = ""
    else:
        Landuse_OSM = landuse_osm_poly[landuse_osm_poly['osm_id'].isin(df["landuse_osm_poly"].iloc[i])][["fclass", "name"]][:5].values.tolist()
    if len(df["pois_osm_poly_elem"].iloc[i])==0:
        Pois_OSM = ""
    else:
        Pois_OSM = pois_osm_poly_elem[pois_osm_poly_elem['osm_id'].isin(df["pois_osm_poly_elem"].iloc[i])][["fclass", "name"]][:5].values.tolist()
    if len(df["natural_osm_poly_elem"].iloc[i])==0:
        natur_osm_poly_elem = ""
    else:
        natur_osm_poly_elem = natural_osm_poly_elem[natural_osm_poly_elem['osm_id'].isin(df["natural_osm_poly_elem"].iloc[i])][["fclass", "name"]][:5].values.tolist()
    if len(df['Worship_osm_point_elem'].iloc[i])==0:
        Worsh_osm_poly_elem = ""
    else:
        Worsh_osm_poly_elem = Worship_osm_point_elem[Worship_osm_point_elem['osm_id'].isin(df['Worship_osm_point_elem'].iloc[i])][["fclass", "name"]][:5].values.tolist()
    
    left_col_colour = "#2A799C"
    right_col_colour = "#C5DCE7"
    html = """<!DOCTYPE html>
<html>

<head>
<h4 style="margin-bottom:0"; width="300px">{}</h4>""".format(Post_code) + """

</head>
    <table style="height: 126px; width: 300px;">
<tbody>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Number of Slot Available</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Number_of_Vehicles) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Connector Types</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Connector_Type) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Connector Power</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Connector_KW) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Worship Area</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Worship_OSM) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Landuse Area</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Landuse_OSM) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">point of Interest Area</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Pois_OSM) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Natural Area</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(natur_osm_poly_elem) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Worship Points</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Worsh_osm_poly_elem) + """
</tr>
</tbody>
</table>
</html>
"""
    return html

In [52]:
def fancy_html_road(distop, durattion):
    Lenght = float("{:.2f}".format(distop))
    expected_time = str(timedelta(seconds=durattion)).split(".")[0]
    consume = consumption
    produced = produced_co2   
    save = saving
    left_col_colour = "#569c2a"
    right_col_colour = "#e0e7c5"
    html = """<!DOCTYPE html>
<html>

<head>
<h4 style="margin-bottom:0"; width="300px">{} KM</h4>""".format(Lenght) +  """

</head>
    <table style="height: 126px; width: 300px;">
<tbody>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Expected Time </span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(expected_time) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Consumed Energy (KWh) </span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(consume) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Produced Co2 (Kg) </span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(produced) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Reduced Co2 (Kg) </span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(save) + """
</tr>


</tbody>
</table>
</html>
"""
    return html

In [53]:
def fancy_html_origin_destination(dictin):
    name = dictin["name"]
    current_temp = dictin['current_temperature']
#     if ada: 
#         forecasted = = ""
    left_col_colour = "#569c2a"
    right_col_colour = "#e0e7c5"
    html = """<!DOCTYPE html>
<html>

<head>
<h4 style="margin-bottom:0"; width="300px">{} </h4>""".format(name) +  """

</head>
    <table style="height: 126px; width: 300px;">
<tbody>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Current Temperature </span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(current_temp) + """
</tr>

</tbody>
</table>
</html>
"""
    return html

In [54]:
#change fir popup
def Marker_create_station(df,map):
    marker_rapid_close = MarkerCluster(name="Rapid Charge")
    marker_fast_close = MarkerCluster(name="Fast Charge")
    marker_slow_close = MarkerCluster(name="Slow Charge")
    for i in range(0,len(df)):
        html = fancy_html_close(i,df)
        iframe = branca.element.IFrame(html=html,width=400,height=300)
        popup = folium.Popup(iframe,parse_html=True)
        if df.iloc[i]['Charger_Category'] == "Rapid" :
            folium.Marker(
            location=[df.iloc[i]['latitude'], df.iloc[i]['longitude']],
            icon=folium.Icon(color='red', icon='plug', prefix='fa'),
            popup=popup,
            ).add_to(marker_rapid_close)
        if df.iloc[i]['Charger_Category'] == "Fast" :
            folium.Marker(
            location=[df.iloc[i]['latitude'], df.iloc[i]['longitude']],
            icon=folium.Icon(color='blue', icon='plug', prefix='fa'),
            popup=popup,
            ).add_to(marker_fast_close)
        if df.iloc[i]['Charger_Category'] == "Slow" :
            folium.Marker(
            location=[df.iloc[i]['latitude'], df.iloc[i]['longitude']],
            icon=folium.Icon(color='lightgray', icon='plug', prefix='fa'),
            popup=popup,
            ).add_to(marker_slow_close)
        marker_rapid_close.add_to(map)
        marker_fast_close.add_to(map)
        marker_slow_close.add_to(map)

In [55]:
def point_to_road(point, df):
    tempor_df=df[(point.coords[0][1]+0.01 > df.latitude) &(point.coords[0][1]-0.01 < df.latitude)
          &(point.coords[0][0]+0.01 > df.longitude)& (point.coords[0][0]-0.01 < df.longitude)].reset_index(drop=True)
    max_distance = 5
    for j , k in tempor_df.iterrows(): 
        near_p = nearest_points(k.geometry,point)
        distance_km =Haversine(near_p[0].coords[0], point.coords[0]).km
        if distance_km <= max_distance:
            max_distance = distance_km
            start_node=k['startNode']
    return start_node

In [56]:
def point_to_a_road(line , max_distance):
    temper = []
    for j , k in scotland_stations_geo_last.iterrows(): 
        near_p = nearest_points(k.geometry,line)
#        print(near_p[1].coords[0])
        distance_km =Haversine(near_p[1].coords[0], k.geometry.coords[0]).km
#        print(distance_km)
        if distance_km <= max_distance:
            temper.append(j)
    if len(temper) != 0:
        closest_df=scotland_stations_geo_last.iloc[temper]
    else: 
        closest_df = pd.DataFrame()
    return closest_df.reset_index(drop=True)

In [57]:
def car_info(car_name, weather):
    if weather == "mild":
        car_range = float(cars_info_df[cars_info_df["car_name"] == car_name].reset_index(drop=True).RRE_Combined_Mild_Weather*1000)
        consumption= float(cars_info_df[cars_info_df["car_name"] == car_name].reset_index(drop=True).REC_Combined_Mild_Weather/1000)
    if weather == "cold":
        car_range = float(cars_info_df[cars_info_df["car_name"] == car_name].reset_index(drop=True).RRE_Combined_Cold_Weather*1000)
        consumption=float(cars_info_df[cars_info_df["car_name"] == car_name].reset_index(drop=True).REC_Combined_Cold_Weather/1000)
    return car_range, consumption 

In [58]:
def emission_calculation(Country, Year, length,car_name = 'Tesla Model S Plaid',benchmark_model= 'A6 QUATTRO',consumed=0):
    if consumed == 0:
        print(cars_info_df[cars_info_df["car_name"] == car_name].reset_index(drop=True)['consumption_Value'])
        consumed_kwh= float(cars_info_df[cars_info_df["car_name"] == car_name].reset_index(drop=True)['consumption_Value'] * length /1000)
    else:
        consumed_kwh= float(consumed * length)
    consumed_kwh= float("{:.2f}".format(consumed_kwh))
    
    ghg_intensity = float(emission_df[(emission_df['Member State:text']== Country) & (emission_df['Date:year']== Year)].reset_index(drop = True)["GHG intensity:number"])
    Co2_produced = float("{:.2f}".format((consumed_kwh *  ghg_intensity)/1000))
    equivalent = combustion_cars[(combustion_cars['Model']==benchmark_model) & (combustion_cars['Cylinders']== 4)]["CO2 Emissions(g/km)"].mean()
    combustion_engine =float("{:.2f}".format((equivalent * length)/1000)) 
    saving = float("{:.2f}".format(combustion_engine - Co2_produced)) # KG CO2
    return consumed_kwh, Co2_produced ,saving

# Reading Data

In [59]:
# to avoid to call Road.shp file which cause unnecessary waiting, datasets called one by one
'''
but call all .shp file in a local document following code can be used. 

#path_osm='C:\\Users\\yusuf\\Downloads\\Dissertation Code\\Open_Data_Geofabric\\'
#all_files_osm = glob.glob(path_osm + "/*.shp")
#all_files_osm
# list_of_dfs_osm = [gpd.read_file(filename).to_crs(epsg=4326) for filename in all_files_osm] # I added to_crs part
# list_of_filenames_osm = [find_between(filename, "Open_Data_Geofabric\\",".shp") for filename in all_files_osm]
# dfnames_and_dfvalues_osm = dict(zip(list_of_filenames_osm, list_of_dfs_osm))
'''
natural_osm_poly = gpd.read_file('~gis_osm_natural_a_free_1.shp').to_crs("EPSG:4326")
natural_osm_point = gpd.read_file('~gis_osm_natural_free_1.shp').to_crs("EPSG:4326")
pois_osm_poly = gpd.read_file('~gis_osm_pois_a_free_1.shp').to_crs("EPSG:4326")
pois_osm_point = gpd.read_file('~gis_osm_pois_free_1.shp').to_crs("EPSG:4326")
Worship_osm_poly = gpd.read_file('~gis_osm_pofw_a_free_1.shp').to_crs("EPSG:4326")
Worship_osm_point = gpd.read_file('~gis_osm_pofw_free_1.shp').to_crs("EPSG:4326")
landuse_osm = gpd.read_file('~gis_osm_landuse_a_free_1.shp').to_crs("EPSG:4326")

In [60]:
natural_osm_poly_elem = natural_osm_poly[natural_osm_poly.name.notnull()].reset_index(drop = True)
natural_osm_point_elem = natural_osm_point[natural_osm_point.name.notnull()].reset_index(drop = True)
pois_osm_poly_elem = pois_osm_poly[pois_osm_poly.name.notnull()].reset_index(drop = True)
pois_osm_point_elem = pois_osm_point[pois_osm_point.name.notnull()].reset_index(drop = True)
Worship_osm_point_elem = Worship_osm_point[Worship_osm_point.name.notnull()].reset_index(drop = True)
Worship_osm_poly_elem = Worship_osm_poly[Worship_osm_poly.name.notnull()].reset_index(drop=True)
landuse_osm_poly = landuse_osm[landuse_osm.name.notnull()].reset_index(drop=True)

In [61]:
method_to_read_list = lambda x: x.strip("[]").replace("'","").split(", ")
scotland_stations_last = pd.read_csv('Scotland_Stations_Combined_Last_4_Numbers.csv',converters={"Worship_osm_poly_elem": method_to_read_list,
                                                                                   "landuse_osm_poly": method_to_read_list,
                                                                                   "pois_osm_poly_elem": method_to_read_list,
                                                                                   "natural_osm_poly_elem": method_to_read_list,
                                                                                   'Worship_osm_point_elem': method_to_read_list,
                                                                                    'natural_osm_point_elem':method_to_read_list})
geometry = [Point(xy) for xy in zip(scotland_stations_last.longitude, scotland_stations_last.latitude)]
scotland_stations_geo_last = GeoDataFrame(scotland_stations_last, crs="EPSG:4326", geometry=geometry)

In [62]:
emission_df = pd.read_csv("co2-emission-intensity-8.csv")
cars_info_df= pd.read_excel("Ev_Cars_Basic.xlsx")
emission_df = emission_df[emission_df["GHG intensity:number"].notna()].reset_index(drop =True)
emission_df=emission_df[emission_df["GHG intensity:number"]!='None'].reset_index(drop =True)
cars_info_df['car_name'] = cars_info_df.car_name.apply(lambda x: x.strip())
cars_info_df['Brand_Name'] = cars_info_df.Brand_Name.apply(lambda x: x.strip())
combustion_cars=pd.read_csv("CO2 Emissions_Canada.csv")
combustion_cars["Make"]=combustion_cars.Make.apply(lambda x: x.strip())
combustion_cars["Model"]=combustion_cars.Model.apply(lambda x: x.strip())

# Routing Algorith

In [127]:
while True:
    start_point= input("Enter Starting Post Code or City Name ")
    if not start_point:
        print('Incorrect input, please try again "Compulsory Field"!')
        continue
    else:
        break
while True:
    end_point= input("Enter Destination ")
    #q=48.8567,2.3508 coordinates should be given like this
    if not end_point:
        print('Incorrect input, please try again "Compulsory Field"!')
        continue
    else:
        break


# day= input("When Are you going? ")
# hour = input("What Time Are You Going? ")
# if not day:
#     day = 0
# else:
#     day = int(day)
# if not hour:
#     hour = 0
# else:
#     hour = int(hour)
# #if check_postcode_validation(start_point):
day = 0
hour = 0 
w_dict= call_Weather_Data_WRT_Day(day,start_point, weather_api_key,hour)
w_dict_end= call_Weather_Data_WRT_Day(day,end_point, weather_api_key,hour)
    #call_Weather_Data_WRT_Day()
#else: #False cannot enter
    #("Post Code Cannot Be Validated")

Enter Starting Post Code or City Name 55.9533,-3.1883
Enter Destination 55.8642,-4.2518


In [128]:
stop_adding = input("Are you adding stop? (Yes or No) \n")
if stop_adding =="Yes" :
    while True:
        stop1 = input("Enter Stop Information (Postcode, City name or Lat,Long Dist")
        if not stop1:
            print('Incorrect input, please try again "Compulsory Field"!')
            continue
        else:
            break    
    w_dict_stop = call_Weather_Data_WRT_Day(day,stop1, weather_api_key,hour)
else: 
    stops= {} 

Are you adding stop? (Yes or No) 
No


In [65]:
Dean_Village = [55.9523 , -3.2182] #Lat , Long
Edin_Castle = [55.9486, -3.1999]
Meadows = [55.9413, -3.1919]
Cairngorms = [57.0492,-3.5616]
Edinburgh_center = [55.9533,-3.1883]
Glasgow_center = [55.8642,-4.2518] 
Loch_lomond = [56.2376,-4.6188]
Smoo_cave= [58.5631, -4.7200]
Glen_coe_waterfall = [56.662254,-4.966097]
Falkirk_wheel = [56.0003, -3.8417]
Dunrobin_castle = [57.9853,-3.9480]
isle_of_skye = [57.3333,-6.2000]

-----> Elevations (Only for len(coordinates) < 200)

In [66]:
# if len(coordinates) <500:
#     body = {"format_in":"polyline","geometry":coordinates}

#     headers = {
#         'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
#         'Authorization': '5b3ce3597851110001cf6248064d2ed997ec4de6be4704d076e293ff',
#         'Content-Type': 'application/json; charset=utf-8'
#     }
#     call = requests.post('https://api.openrouteservice.org/elevation/line', json=body, headers=headers)
#     print(call.status_code, call.reason)
#     #print(call.json()["geometry"]["coordinates"])

#     elevations =[i[2] for i in call.json()["geometry"]["coordinates"]]
#     plt.plot(elevations)
#     plt.xlabel("Points in the line")
#     plt.ylabel(f"Elevation (in meter)")
# else: 
#     print(f"the line includes more nodes ({len(coordinates)}) than the maximum limit (200)")

<----Elevations (Only for len(coordinates) < 200)

In [129]:
scotland_5_days_tour = {"Aviemore": Point(-3.833476, 57.191518),
                       "Ben_Nevis": Point(-5.113181, 56.816968), 
                       "Loch_ness": Point(-4.564502, 57.230575),
                       "Glen_coe_waterfall": Point(-4.966105, 56.662282), 
                       "Loch_lomond": Point(-4.386566, 56.182894),
                       "Loch_Katrine": Point(-4.588534, 56.25105)}
angus_coastal = {"Arbroath Abbey": Point(-2.5824,56.5622), # Start = "V&A Dundee": Point(-2.9670,56.4575)
                "House of Dun": Point(-2.5408,56.7293),
                } # Destination "Dunnottar Castle": Point(-2.1972 ,56.9458)
# w_dict = {}
# w_dict_end = {}
# w_dict["longitude"] = Edinburgh_center[1] #-3.4092
# w_dict["latitude"] = Edinburgh_center[0] #55.9534
# w_dict_end["longitude"] = Glasgow_center[1] #-3.2990
# w_dict_end["latitude"] = Glasgow_center[0] #55.9198
star_point_geo = Point(w_dict["longitude"], w_dict["latitude"])
end_point_geo= Point(w_dict_end["longitude"], w_dict_end["latitude"])
#stops = {"first": Point(-3.1919,55.9413)}
#stops = scotland_5_days_tour
wall_times=[]

In [132]:
%%time
for _ in range(50):
    start = time.time()
    if bool(stops):
        return_api_json = call_OSMR_route_add_stop(star_point_geo,end_point_geo,stopss = stops)
    else:
        return_api_json = call_OSMR_route(star_point_geo,end_point_geo)

    distance =return_api_json['routes'][0]['distance']/1000
    exp_time = return_api_json['routes'][0]['duration']
    coordinates = return_api_json['routes'][0]['geometry']['coordinates']
    changed = [[p[1], p[0]] for p in coordinates]
    consumption,produced_co2, saving =emission_calculation('United Kingdom', 2019, distance,"Tesla Model 3 Performance","320i")#'Tesla Model S Plaid'

    if len(coordinates) <500:
        body = {"format_in":"polyline","geometry":coordinates}

        headers = {
            'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
            'Authorization': #openrouteservice API_KEY,
            'Content-Type': 'application/json; charset=utf-8'
        }
        call = requests.post('https://api.openrouteservice.org/elevation/line', json=body, headers=headers)
        print(call.status_code, call.reason)
        #print(call.json()["geometry"]["coordinates"])

        elevations =[i[2] for i in call.json()["geometry"]["coordinates"]]
    line = LineString(coordinates)
    closest_stations=point_to_a_road(line,0.5)

    dist_station=closest_stations[['station_id','geometry']].reset_index(drop=True)
    dist_station_types= closest_stations[['station_id',"Charger_Category"]].reset_index(drop=True)
    node_list = ['Start_point','End_point']

    geo_list = [star_point_geo,end_point_geo]
    if bool(stops):
        for key,value in stops.items():
            node_list.insert(len(node_list),key)
            geo_list.insert(len(geo_list),value)       #a.insert(len(a),5)
    route_info = pd.DataFrame(
        {'station_id': node_list,
         'geometry': geo_list,
        })
    df_with_nearby = pd.concat([route_info,dist_station]).reset_index(drop=True)
    all_node_list = [i for i in df_with_nearby.geometry.apply(lambda x: [x.coords[0][0],x.coords[0][1]])]
    df_matrix_all = call_distance_matrix_openroute(all_node_list)



    G_all = nx.from_pandas_edgelist(df_matrix_all, source='Origin', target='Dest',edge_attr=['Duration', 'Distance'],create_using=nx.DiGraph())
    energy=nx.floyd_warshall_numpy(G_all, weight='Distance')
    durations=nx.floyd_warshall_numpy(G_all, weight='Duration')
    #car_rangee , car_consumption =car_info("Tesla Model 3 Performance","cold")#'Tesla Model S Plaid'
    if ((w_dict["current_temperature"]+ w_dict_end["current_temperature"])/2) < 0:
        wet_cond = "cold"
    else:
        wet_cond = "mild"
    print(wet_cond)
    #car_rangee , car_consumption =car_info("Tesla Model 3 Performance",wet_cond)#'Tesla Model S Plaid'
    car_rangee , car_consumption =car_info("Nissan Leaf",wet_cond)#'Tesla Model S Plaid'
    chargers = []
    for i,v in dist_station_types.iterrows():
        if v.Charger_Category == "Rapid":
            chargers.append({"node_id": i+len(node_list),"cs_type": 0})
        if v.Charger_Category == "Fast":
            chargers.append({"node_id": i+len(node_list),"cs_type": 1})
        if v.Charger_Category == "Slow":
            chargers.append({"node_id": i+len(node_list),"cs_type": 2})
    battery =round(car_rangee*car_consumption/1000)
    energy_consumption = energy*car_consumption/1000

    tesla_chargers = [
            {
                "cs_type": 0,
                "time": [
                    0.0,
                    300,
                    1800,
                    3600
                ],
                "charge": [
                    0.0,
                    0.2*battery,#car_rangee,
                    0.8*battery,
                    battery
                ]
            },
            {
                "cs_type": 1,
                "time": [
                    0.0,
                    4320,
                    17280,
                    21600
                ],
                "charge": [
                    0.0,
                    0.2*battery,#car_rangee,
                    0.8*battery,
                    battery
                ]
            },
            {
                "cs_type": 2,
                "time": [
                    0.0,
                    14400,
                    57600,
                    72000
                ],
                "charge": [
                    0.0,
                    0.2*battery,#car_rangee,
                    0.8*battery,
                    battery
                ]
            }
        ]
    leaf_chargers = [
            {
                "cs_type": 0,
                "time": [
                    0.0,
                    600,
                    2400,
                    3600
                ],
                "charge": [
                    0.0,
                    0.2*battery,#car_rangee,
                    0.8*battery,
                    battery
                ]
            },
            {
                "cs_type": 1,
                "time": [
                    0.0,
                    5320,
                    16280,
                    21600
                ],
                "charge": [
                    0.0,
                    0.2*battery,#car_rangee,
                    0.8*battery,
                    battery
                ]
            },
            {
                "cs_type": 2,
                "time": [
                    0.0,
                    14400,
                    57600,
                    72000
                ],
                "charge": [
                    0.0,
                    0.2*battery,#car_rangee,
                    0.8*battery,
                    battery
                ]
            }
        ]

    fvr_input = {}
    fvr_input["max_q"] = car_rangee
    #fvr_input["breakpoints_by_type"]= tesla_chargers #leaf_chargers
    fvr_input["breakpoints_by_type"]= leaf_chargers
    fvr_input["css"]= chargers
    fvr_input["energy_matrix"] = energy_consumption #energy
    fvr_input["time_matrix"] = durations

    percentage = 1
    initial_battery=battery*percentage

    route = [0,1]  # route to make energy feasible
    if bool(stops):
        node_value=2
        for key,value in stops.items():
            route.insert(-1,node_value)
            node_value += 1
    q_init =initial_battery #77000
    frvcp_solver = solver.Solver(fvr_input , route, q_init)
    duration, feas_route = frvcp_solver.solve()
    if duration == inf:
        print("There is no available road found, charge your car before the trip")
    else: 
        print(f"Duration: {str(timedelta(seconds=duration))}")
    # Duration: 7.339
        print(f"Energy-feasible route:\n{feas_route}")

    stations_visited = [i[0] for i in feas_route]
    selected_elements = [all_node_list[index] for index in stations_visited]

    consumption_total_road = 0 
    first = 0
    for i in stations_visited[1:]:
        temp_df=df_matrix_all[(df_matrix_all.Origin == first) & (df_matrix_all.Dest == i)].reset_index(drop=True)
    #    print(temp_df)
        consumption_of_miniroad = temp_df.Distance[0]*car_consumption/1000
    #    print(consumption_of_miniroad)
        consumption_total_road += consumption_of_miniroad
    #    print(consumption_total_road)
        first = i

    opt_distance,line2= call_OSMR_optimal_route(selected_elements)
    line2_changed=[[p[1], p[0]] for p in line2]


    m_final = folium.Map(location=[55.9533, 3.1883], zoom_start=5)

    final_distance = opt_distance/1000 #convert km
    #consumption,produced_co2, saving = emission_calculation('United Kingdom', 2019, final_distance,"Tesla Model 3 Performance","320i",car_consumption)#'Tesla Model S Plaid')
    consumption,produced_co2, saving = emission_calculation('United Kingdom', 2019, final_distance,"Nissan Leaf","GOLF",car_consumption)#'Tesla Model S Plaid')
    html = fancy_html_road(final_distance,duration)
    iframe = branca.element.IFrame(html=html,width=400,height=300)
    popup = folium.Popup(iframe,parse_html=True)

    folium.PolyLine(locations=line2_changed,weight=5,popup=popup).add_to(m_final)

    html = fancy_html_origin_destination(w_dict)
    iframe = branca.element.IFrame(html=html,width=400,height=300)
    popup = folium.Popup(iframe,parse_html=True)
    folium.Marker(
                location=[w_dict["latitude"], w_dict["longitude"]],
                icon=folium.Icon(color='black', icon="fa-map-marker", prefix='fa'),#'fa-flag-checkered'
                popup=popup,
                ).add_to(m_final)

    html = fancy_html_origin_destination(w_dict_end)
    iframe = branca.element.IFrame(html=html,width=400,height=300)
    popup = folium.Popup(iframe,parse_html=True)
    folium.Marker(
                location=[w_dict_end["latitude"], w_dict_end["longitude"]],
                icon=folium.Icon(color='black', icon='fa-flag-checkered', prefix='fa'),#'fa-flag-checkered'
                popup=popup,
                ).add_to(m_final)
    if len(stations_visited) > 2:
        set1 = set(stations_visited)
        set2 = set(route)
        missing = list(sorted(set1 - set2))
        visited_station_ids = df_with_nearby.iloc[missing]
        visited_station_list=visited_station_ids["station_id"].to_list()
        passed_stations = closest_stations[closest_stations['station_id'].isin(visited_station_list)].reset_index(drop=True)
        Marker_create_station(passed_stations,m_final)
    if bool(stops):
        for key,value in stops.items():
            folium.Marker(
                location=[value.y, value.x],
                icon=folium.Icon(color='black', icon='street-view', prefix='fa'),#'fa-flag-checkered'
                popup=str(key).replace("_", " "),
                ).add_to(m_final)
    end = time.time()
    wall_times.append(end-start)

0    134
Name: consumption_Value, dtype: int64
200 OK
mild
Duration: 0:59:17.750000
Energy-feasible route:
[(0, None), (1, None)]
0    134
Name: consumption_Value, dtype: int64
200 OK
mild
Duration: 0:59:17.750000
Energy-feasible route:
[(0, None), (1, None)]
0    134
Name: consumption_Value, dtype: int64
200 OK
mild
Duration: 0:59:17.750000
Energy-feasible route:
[(0, None), (1, None)]
0    134
Name: consumption_Value, dtype: int64
200 OK
mild
Duration: 0:59:17.750000
Energy-feasible route:
[(0, None), (1, None)]
0    134
Name: consumption_Value, dtype: int64
200 OK
mild
Duration: 0:59:17.750000
Energy-feasible route:
[(0, None), (1, None)]
0    134
Name: consumption_Value, dtype: int64
200 OK
mild
Duration: 0:59:17.750000
Energy-feasible route:
[(0, None), (1, None)]
0    134
Name: consumption_Value, dtype: int64
200 OK
mild
Duration: 0:59:17.750000
Energy-feasible route:
[(0, None), (1, None)]
0    134
Name: consumption_Value, dtype: int64
200 OK
mild
Duration: 0:59:17.750000
Energy

In [133]:
wall_times

[1.2477035522460938,
 1.2600877285003662,
 1.5132503509521484,
 1.5625271797180176,
 1.5871233940124512,
 1.8063945770263672,
 1.6065690517425537,
 1.5254018306732178,
 1.5357303619384766,
 1.7222001552581787,
 1.517171859741211,
 1.5455725193023682,
 1.7870872020721436,
 1.5272462368011475,
 1.5445806980133057,
 1.5407333374023438,
 1.6287522315979004,
 1.578139066696167,
 1.52156400680542,
 1.6557302474975586,
 1.8711462020874023,
 1.505857229232788,
 1.525644302368164,
 1.525939702987671,
 1.5362653732299805,
 1.5021347999572754,
 1.5374367237091064,
 1.5210988521575928,
 1.5338480472564697,
 2.0937530994415283,
 1.5459816455841064,
 1.537003993988037,
 1.533287525177002,
 1.5054914951324463,
 1.532823085784912,
 1.5075139999389648,
 1.5277082920074463,
 1.535006046295166,
 1.520766258239746,
 1.6777920722961426,
 1.545391321182251,
 1.6522722244262695,
 1.550321340560913,
 1.5153343677520752,
 1.5568296909332275,
 1.5273656845092773,
 1.5647547245025635,
 1.5294396877288818,
 1.549

In [749]:
#m_final.save('Edinburgh_to_Glasgow_Centers_Nissan_chargin_only_fast_001%.html')